In [ ]:
import twarc
import os
import pandas as pd

In [ ]:
#names of the organizations, may replace with needed Banks, NGOs, Airlines
s="""@ICRC_Nairobi
@ICRC_library
@RedCrossLebanon
@RedCross
@BritishRedCross
@RedCrossAU
@irishredcross
@philredcross
@IFRC_Europe
@NepalRedCross"""

In [ ]:
#Dwonload all tweets of the organisation
fscript=open("download/download.sh",'w')
for ind,org in enumerate(s.split('\n')):
    fngo=open("download/ngos{0}.txt".format(ind),'w')
    fngo.write(org.replace("@",""))
    fngo.close()
    fscript.write("twarc2 timelines --use-search download/ngos{0}.txt download{1}.jsonl\n".format(ind,org.replace("@","")))
fscript.close()

In [ ]:
# %sh download.sh

In [ ]:
#dehydrate downloaded tweets to get their ids in case of further need
fscript=open("dehydrate/dehydrate.sh",'w')
for file in os.listdir('download'):
    if file.split('.')[1]=='jsonl':
        fscript.write("twarc2 dehydrate download/{0} dehydrate/{1}.txt\n".format(file,file.split('.')[0]))
        # fscript.write("twarc2 conversations --archive {0}.txt > {1} \n".format(file.split('.')[0]))
fscript.close()

In [ ]:
# %sh dehydrate.sh

In [ ]:
#download back tweets based on their ids
fscript=open("hydrate/hydrate.sh",'w')
for file in os.listdir('dehydrate'):
    if file.split('.')[1]=='txt':
        fscript.write("twarc2 hydrate dehydrate/{0} hydrate/{1}.json\n".format(file,file.split('.')[0]))
        # fscript.write("twarc2 conversations --archive {0}.txt > {1} \n".format(file.split('.')[0]))
fscript.close()

In [ ]:
# %sh hydrate.sh

In [ ]:
#process the tweets to get the conversation_ids
for file in os.listdir('hydrate'):
    if file.split('.')[1]=='json':
        df=pd.read_json('hydrate/'+file,lines=True)
        conversation_ids=[]
        for line in df['data']:
            for element in line:
                conversation_ids.append(element['id'])
        for line in df['includes']:
            for element in line["tweets"]:
                conversation_ids.append(element['conversation_id'])
        print(len(set(conversation_ids)))
        with open("hydrate/{0}.txt".format(file.split('.')[0]), 'w') as fp:
            for id in set(conversation_ids):
                fp.write("%s\n" % id)

In [ ]:
#download the conversations
fscript=open("conversations/conversations.sh",'w')
for file in os.listdir('hydrate'):
    if file.split('.')[1]=='txt':
        fscript.write("twarc2 conversations --archive hydrate/{0} conversations/{1}.json\n".format(file,file.split('.')[0]))
fscript.close()

In [ ]:
# %sh conversations.sh